In [1]:
import pandas as pd
import pandas as pd
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
from string import digits
import numpy as np

In [2]:
import nltk
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [3]:
def preprocess_data(s):
    s = s.lower()
    data = re.sub(r'[^\x00-\x7F]+', ' ', s)
    final_str = data.translate(None, string.punctuation)
    filter_str = final_str.translate(None, digits)
    nltk_tokens = nltk.word_tokenize(filter_str)
    #Next find the roots of the word
    str_= ''
    for w in nltk_tokens:

        if w not in stopwords.words('english'):
            str_ += ' '  + (lemmatizer.lemmatize(w))
    
    return str_.strip()

In [4]:
df_train = pd.read_csv('/home/prashant/Downloads/c3cc8568-0-dataset/train.csv')
df_train['tag'] = 'train'
print df_train.shape

(43266, 10)


In [ ]:
df_test = pd.read_csv('/home/prashant/Downloads/c3cc8568-0-dataset/test.csv') 
df_test['tag'] = 'test'
print df_test.shape

In [ ]:
df = pd.concat([df_train , df_test],axis= 0)
print df.shape
df.head()

In [ ]:
#df_train.isnull().sum()
df['Complaint-reason'] = df['Complaint-reason'].apply(preprocess_data)

In [ ]:
cr = dict(df['Complaint-reason'].value_counts())
l = []
for r in cr:
    if cr[r] < 300:
        l.append(r)
len(l)

In [ ]:
df = df[['Complaint-ID','tag','Complaint-Status','Transaction-Type','Consumer-disputes']]

In [ ]:
df.head()

In [ ]:
tran_type = {'Debt collection' : 'd' ,
             'Credit reporting':'c' ,
             'Credit card':'c',
             'Credit reporting, credit repair services, or other personal consumer reports':'c',
             'Credit card or prepaid card':'c',\
             'Prepaid card':'c',
             'Mortgage':'l',
             'Student loan':'l',
             'Consumer Loan':'l',
             'Payday loan':'l',
             'Vehicle loan or lease':'l',\
             'Payday loan, title loan, or personal loan':'l',\
             'Virtual currency':'c',\
             'Money transfer, virtual currency, or money service':'c',\
             'Money transfers':'b',\
             'Bank account or service':'b',\
             'Checking or savings account':'b',
             'Other financial service':'b'
    }
len(tran_type)
replace = []
for t in df['Transaction-Type']:
    replace.append(tran_type[t])
    
df['Transaction-Type'] = replace

In [ ]:
df.head()

In [ ]:
df_cat = df[['Transaction-Type','Consumer-disputes']]
df_one = pd.get_dummies(df_cat)


In [ ]:
df_one['Complaint-ID'] = df['Complaint-ID']
df_one['tag'] = df['tag']

In [ ]:
df_one['Complaint-Status'] = df['Complaint-Status']


In [ ]:
complete_df_one_hot = df_one

train_data = complete_df_one_hot.loc[complete_df_one_hot['tag']== 'train']
train_data.drop(['tag'],inplace=True,axis=1)
test_data = complete_df_one_hot.loc[complete_df_one_hot['tag']== 'test']
test_data.drop(['tag'],inplace=True,axis =1)

In [ ]:
y_train = train_data['Complaint-Status']
x_train = train_data.drop(['Complaint-Status','Complaint-ID'],axis =1)

In [ ]:
test_data.head()

In [ ]:
#x_train_one_hot = pd.get_dummies(x_train,prefix=['Complaint-reason'])


In [ ]:
#y_train_one_hot = pd.get_dummies(y_train,prefix=['Complaint-Status'])

In [ ]:
# print y_train_one_hot.shape
# print x_train_one_hot.shape

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.linear_model import LogisticRegression
X_train, X_test, y_train_internal, y_test_internal = train_test_split(x_train,y_train ,random_state = 0,)

In [ ]:
nb = MultinomialNB().fit(x_train, y_train)
pred_internal = nb.predict(X_test)
import sklearn
sklearn.metrics.f1_score(pred_internal,y_test_internal,average='weighted')

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(n_estimators = 100, random_state = 42)

# Train the model on training data
rm = rf.fit(X_train, y_train_internal)
pred_internal = rm.predict(X_test)

In [ ]:
import sklearn
sklearn.metrics.f1_score(pred_internal,y_test_internal,average='weighted')

In [ ]:
id_ = list(test_data['Complaint-ID'])

In [ ]:
test_data.head()

In [ ]:
print train_data.shape
test_data.shape
test_data.drop(['Complaint-Status','Complaint-ID'],inplace=True,axis=1)

In [ ]:
test_data.head()
y_pred = nb.predict(test_data)

In [ ]:
result = pd.DataFrame(id_,columns=['Complaint-ID'])
result['Complaint-Status'] = y_pred

In [ ]:
result.to_csv('/home/prashant/Downloads/c3cc8568-0-dataset/one_hot_ign_3.csv',index=None)